<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/Distrib_vs_Single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/cagBRT/PySpark.git



---



---



# Setup Spark

In [ ]:
!pip install pyspark

In [ ]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [ ]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [ ]:
spark = (
    SparkSession.builder
        .appName('learn')
        .config('spark.driver.memory', '8g')
        .master('local[4]',)
        .config('spark.sql.execution.arrow.pyspark.enabled', True)
        .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False)
        .getOrCreate()
)

In [ ]:
sc=spark.sparkContext.getOrCreate()



---



---



# Get all of Shakespeare’s works

In [ ]:
import time

**Let's find out how long it takes a distributed computer to perform the task of counting all the words in Shakespeare's works**.

In [ ]:
start_time = time.time()
#Count the number of words in all of Shakespear's works
Words=sc.textFile("/content/PySpark/shakespeare.txt")
WordsCount=Words.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
WordsCount.count()

**Let's find out how long it takes a *single computer* to perform the task of counting all the words in Shakespeare's works**.

In [ ]:
start_time = time.time()
f= open("/content/PySpark/shakespeare.txt", "r")
words_shakes= f.read()
f.close()
word_shakes_python=words_shakes.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(word_shakes_python)

In [ ]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

In [ ]:
start_time = time.time()
#Count the number of distinct words
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
DistinctWordsCount.count()

In [ ]:
start_time = time.time()
words_unique=set(word_shakes_python)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

In [ ]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

In [ ]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
SortedWordsCount.top(20)

In [ ]:
start_time = time.time()
counted_words=[]
words_unique=list(words_unique)
for i in range(len(words_unique)):
  tuples= (word_shakes_python.count(words_unique[i]), words_unique[i])
  counted_words.append(tuples)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time= time.time() - start_time

In [ ]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)